# Discussion of Lag-Adjusted Spearman Rank Correlation
** Derivation of the lag-adjusted Spearman Rank Correlation Test **

### Motivation
Since we know that if wastewater injection induces seismicity, the effect of injection on seismicity will occur at some lag $\geq$ 0 months. If the lag is > 0 (e.g. if wastewater injection causes increased seismicity at least 1 month in the future), then we expect seismicity to depend on past water injection. The purpose of creating adjusting the Spearman Rank Correlation by a particular lag is to quantify correlation if it occurs at lag > 0. 

### The Test
Spearman's rank correlation can be proved to be defined as follows: 

$$r_s = 1 - \frac{6D}{N^3-N}$$ 

where $r_s$ is the rank correlation, $N$ is the total number of observations in the sample and $D$ is defined as follows: 

$$ D = \frac{1}{3}N(N+1)(2N+1) - 2\sum{iT_i} $$ 

where $i$ is defined as the index of one of the lists and $T_i$ is the rank of observation $i$ in one of the lists. 

Therefore, a statistical test could be defined where the null hypothesis $H_0$ is $r_s \neq 0$. Since the only non-constant in the equation is $\sum{iT_i}$, it would be sufficient to reject $H_0$ for large values of $\sum{iT_i}$. To generate a p-value for this test, the original $\sum{iT_i}$ can be compared to all permutations of $T_i$ (since under the null hypothesis it would be equally likely to see any rank $T_i$ to be paired with any $i$). 

Adjusting this test for different lags would simply mean shifting the two lists of ranks by some lag $k$ and running the above method with $N-k$ observations. However, since N changes with the lag, it is easiest to simply compute $r_s$, the Spearman rank correlation, as the test statistic. 

### Simple Examples
In this section, we present simple numerical examples using the Spearman Rank Correlation Test and our lag-adjusted SRCT test. First, we begin with two completely linearly dependent lists. Then, we change the dependency by introducing a lagged dependency with lag correlation = 1. 

**Two completely linearly dependent lists**

In [111]:
# import relevant modules
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.stats import rankdata
from scipy.stats import spearmanr
from sympy.utilities.iterables import multiset_permutations
from statsmodels.stats import multitest

# create our two dependent lists
x = np.array(list(range(1,11,2))) # 1-11 odds
y = np.array(x+1) # y = x+1

# convert to ranks
x_ranked = rankdata(x)
y_ranked = rankdata(y)

In [124]:
# get dot product
xy_dot = np.dot(x_ranked, y_ranked)

# get an array of all possible dot products when shuffling one rank
x_ranked_perm = multiset_permutations(x_ranked)

# for each permutation, calculate and store dot product
xy_dot_perm = list()
for perm in x_ranked_perm:
    xy_dot_perm.append(np.dot(perm, y_ranked))
    
# compare original dot product to all permutations
p_value = sum(xy_dot_perm >= xy_dot) / len(xy_dot_perm)
print("p-value: ", p_value)

p-value:  0.00833333333333


** Two linearly dependent lagged lists **

In [21]:
# create our two dependent (with lag = 2) lists
x = np.array(list(range(1,15,2))) # 1-15 odds
y = np.append([0,0], list(x[0:len(x)-2]+1))  # y = x_{t-2} + 1

# taking the lag dot product is equivalent to shifting the lists
x = x[:5]
y = y[2:]

# convert to ranks
x_ranked = rankdata(x)
y_ranked = rankdata(y)

In [22]:
# get dot product
xy_dot = np.dot(x_ranked, y_ranked)

# get an array of all possible dot products when shuffling one rank
x_ranked_perm = multiset_permutations(x_ranked)

# for each permutation, calculate and store dot product
xy_dot_perm = list()
for perm in x_ranked_perm:
    xy_dot_perm.append(np.dot(perm, y_ranked))
    
# compare original dot product to all permutations
p_value = sum(xy_dot_perm >= xy_dot) / len(xy_dot_perm)
print("p-value: ", p_value)

p-value:  0.00833333333333


### More complicated cases
If the lag dependency is known, it has been shown to be easy to calculate the appropriate SRCT. However, what if the lag is not known? This section will discuss techniques for this case. Code and numerical examples follow.

** Method 1: Compare the maximum normalized dot products across each permutation **

1. Compute the Spearman rank correlation (rho) for every lag in a certain window (say, $k = 0,1,...,12$).
2. Take the maximum.
3. Repeat (1)-(2) for each permutation to form a list of length $N!$.
4. Compare the original rho to the list generated in (3) to find a p-value.

** Method 2: Search for optimal lag by computing p-values for each lag **

1. For each lag in a certain window, compute the Spearman rank correlation.
2. Repeat (1) for each permutation.
3. Compute p-values for each lag. 
4. Use the Holm–Bonferroni method to compare p-values. 

In [126]:
def spearman_lag(list1, list2, lag=0, use_zeros=False):
    """
    Shifts `list2` backward by `lag` number of entries.
    Removes the tail ends of the rank vectors that fall off
    after shifting. Then returns the Spearman rank correlation.
    For instance, suppose
         list1  = [1, 2, 3, 4]
         list2  = [4, 3, 2, 1]
         lag     = 2.
    After shifting and removing the fall off, we have
         list1' = [1, 2]
         list2' = [2, 1]

    Assumes that len(list1) == len(list2) and all ranks
    are non-negative.

    Args:
        list1    (1D np.array) : First dataset
        list2    (1D np.array) : Second dataset
        lag       (int)         : Number of entries to shift
        use_zeros (bool)        : Use zeros in rank vectors
                                  for normalizing
    Returns:
        Spearman's rho coefficient
    """
    assert len(list1) == len(list2), \
           "Vectors are not the same length"

    # Count the number of entries that would be zeroed out
    # in either vector
    num_zeros = 0
    if not use_zeros:
        num_zeros = np.sum((list1 * list2) == 0)

    # Shift vectors
    list1_shift = list1[:(len(list2)-lag)]
    list2_shift = list2[lag:]
    
#     # Compute Spearman's rho coefficient
#     N = len(ranks1_shift)
#     D = (1/3) * N * (N+1) * (2*N+1) - 2*np.dot(ranks1_shift, ranks2_shift)
#     rho = 1 - (6*D)/(N**3 - N)
    
    rho = spearmanr(list1_shift, list2_shift)[0]
    
    return rho

def largest_rho(list1, list2, min_lag=0, max_lag=12, use_zeros=False):
    """
    Computes largest rho for all lags specified by `min_lag` and `max_lag` inclusive. 
    """
    rhos = [spearman_lag(list1, list2, lag, use_zeros) \
                for lag in range(min_lag, max_lag + 1)]
    return np.amax(rhos)

def all_rho(list1, list2, min_lag=0, max_lag=12, use_zeros=False):
    """
    Computes all rhos for all lags in range(min_lag, max_lag). 
    """
    rhos = [spearman_lag(list1, list2, lag, use_zeros) \
                for lag in range(min_lag, max_lag + 1)]

    return rhos

def simulate(list1, list2, method, min_lag=0, max_lag=12, use_zeros=False):
    """

    """
        
    # Get all permutations of list1
    list1_perm = multiset_permutations(list1)
    
    # Change output depending on `method`
    if method == 1:
        
        # Store original largest rho
        original_rho = largest_rho(list1, list2, min_lag, max_lag, use_zeros)
    
        # Compute maximum rho for each permutation
        largest_rhos = list()
        for perm in list1_perm:
            largest_rhos.append(largest_rho(perm, list2, min_lag, max_lag, use_zeros))
        
        # Return a p-value
        return sum(largest_rhos >= original_rho) / len(largest_rhos)
            
    if method == 2:
        
        # Store original rhos for each lag
        original_rhos = all_rho(list1, list2, min_lag, max_lag, use_zeros)
        
        # Compute all rhos for each permutation
        all_rhos = list()
        for perm in list1_perm:
            all_rhos.append(all_rho(perm, list2, min_lag, max_lag, use_zeros))
        all_rhos = np.array(all_rhos)
            
        # Compute p-values for each lag
        p_values = list()
        for i in range(len(original_rhos)):
            p_values.append(sum(all_rhos[:,i] >= original_rhos[i]) / len(all_rhos))
#         print(p_values)
        
        # Use the Holm-Sidak method to change to appropriate p-value
        results = multitest.multipletests(p_values, alpha=0.05, method='hs')
        
        return(results)
        
       


#### Numerical example for Method 1

In [127]:
%%time
## Use the same equations from previous lag-2 dependent processes

# create our two dependent (with lag = 2) lists
x = np.array(list(range(1,15,2))) + np.random.normal(0,1,7) # 1-19 odds + epsilon
y = np.append(np.random.normal(0,1,2), list(x[0:len(x)-2]-5)) + \
        np.random.normal(0,1,7)  # y = x_{t-2} - 5 + epsilon

print("Data:")
print(x)
print(y)

print("lag = 0: ", spearman_lag(x, y, lag = 0))
print("lag = 1: ", spearman_lag(x, y, lag = 1))
print("lag = 2: ", spearman_lag(x, y, lag = 2))
print("lag = 3: ", spearman_lag(x, y, lag = 3))

p0_1 = simulate(x, y, method = 1, min_lag=0, max_lag=1)
p0_2 = simulate(x, y, method = 1, min_lag=0, max_lag=2)
p0_3 = simulate(x, y, method = 1, min_lag=0, max_lag=3)
p0_4 = simulate(x, y, method = 1, min_lag=2, max_lag=4)

print(p0_1)
print(p0_2)
print(p0_3)
print(p0_4)

Data:
[  1.23272539   2.2233734    6.66530854   5.60462647   8.03274885
  11.14274239  13.90315368]
[-0.23125583 -0.43821268 -3.29607376 -3.80664347  2.80722196  0.9869344
  2.69114191]
lag = 0:  0.571428571429
lag = 1:  0.314285714286
lag = 2:  0.8
lag = 3:  0.2
0.209920634921
0.112698412698
0.277777777778
0.4
Wall time: 48.6 s


#### Numerical example for Method 2

In [116]:
%%time

## Use the same equations from previous lag-2 dependent processes

# create our two dependent (with lag = 2) lists
x = np.array(list(range(1,15,2))) + np.random.normal(0,1,7) # 1-13 odds + epsilon
y = np.append(np.random.normal(0,1,2), list(x[0:len(x)-2]-5)) + \
        np.random.normal(0,1,7)  # y = x_{t-2} - 5 + epsilon

print("Data:")
print(x)
print(y)

print("lag = 0: ", spearman_lag(x, y, lag = 0))
print("lag = 1: ", spearman_lag(x, y, lag = 1))
print("lag = 2: ", spearman_lag(x, y, lag = 2))
print("lag = 3: ", spearman_lag(x, y, lag = 3))

p0_1 = simulate(x, y, method = 2, min_lag=0, max_lag=1)
p0_2 = simulate(x, y, method = 2, min_lag=0, max_lag=2)
p0_3 = simulate(x, y, method = 2, min_lag=0, max_lag=3)
p0_4 = simulate(x, y, method = 2, min_lag=2, max_lag=4)

print(p0_1)
print(p0_2)
print(p0_3)
print(p0_4)

[0.15119047619047618, 0.087499999999999994]
[0.15119047619047618, 0.087499999999999994, 0.0083333333333333332]
(array([False, False], dtype=bool), array([ 0.16734375,  0.16734375]), 0.025320565519103666, 0.025)
(array([False, False,  True], dtype=bool), array([ 0.16734375,  0.16734375,  0.02479225]), 0.016952427508441503, 0.016666666666666666)
Wall time: 18.1 s
